In [2]:
import vmodel
import os
import numpy as np
import h5py
import datetime
import scipy.spatial


def calc_order(vel: np.ndarray) -> float:
    """Compute order parameter from velocity matrix
    Args:
        vel: velocity matrix (N x D)
    Returns:
        order: velocity correlation
    """
    N, _ = vel.shape
    speed = np.linalg.norm(vel, axis=1, keepdims=True)  # N x 1
    speed_prod = speed.dot(speed.T)  # N x N
    mask = (speed_prod != 0)  # avoid division by zero!
    dot_prod = vel.dot(vel.T)  # N x N
    np.fill_diagonal(dot_prod, 0)  # i != j
    return (dot_prod[mask] / speed_prod[mask]).sum() / (N * (N - 1))

    


out_str = "/extra2/knopf/vmodel_output/noPred_/"
saveLoc = "/extra2/knopf/vmodel_output/"
saveName = "noPred_"


args = {
'nprey': 100,
'npred': 1,
'frange': 10,
'fstr': 0.0,
'visPred': 300.0,
'visPrey': 330,
'astr': 3.0,
'dphi': 0.2,
'repPrey': 3,
'repRadPrey': 1.5,
'repPred': 1,
'repRadPred': 20,
'attPrey': 3,
'attRadPrey': 1.5,
'repCol': 10000000,
'hstr': 1,
'steps': 1500,
    }



paraChange1_name = "astr"
paraChange2_name = "dphi"
steps = 20
reps = 30

total = steps*steps*reps

paraChange1_val = np.linspace(0,3,steps)
paraChange2_val = np.linspace(0,1,steps)

pol_scan = np.zeros((steps, steps))
IID_scan = np.zeros((steps, steps))
CND_scan = np.zeros((steps, steps))

time_now = datetime.datetime.now()
time_elapsed = 0


for i in range(len(paraChange1_val)):
    
    for j in range(len(paraChange2_val)):

        #np.savetxt(str(saveLoc)+""+str(saveName)+"pol_"+str(paraChange1_name)+"_"+str(paraChange2_name)+".csv", pol_scan, delimiter=",")
  
        pol_reps = []
        IID_reps = []
        CND_reps = []
        
        args[paraChange1_name] = paraChange1_val[i]
        args[paraChange2_name] = paraChange2_val[j]

        npred = args["npred"]
        nprey = args["nprey"]

        args_str = '_'.join(f'{k}_{v}' for k, v in args.items())

        file_h5 = f'{out_str}_{args_str}.states.nc'
        #file_h5 = f'/home/lars/vmodel_output/_nprey_100_npred_1_frange_10_fstr_0.0_visPred_90.0_visPrey_330_astr_3_dphi_0.2_repPrey_3_repRadPrey_1_repPred_1_repRadPred_20_attPrey_3_attRadPrey_1_repCol_10000000_hstr_1_steps_50000.states.nc'

        #print(file_h5)
        


        try:
            with h5py.File(file_h5) as fh5:
                vel = np.moveaxis(np.array(fh5['/velocity']), [3,2], [1,3])[:,:,:,:]
                pos = np.moveaxis(np.array(fh5['/position']), [3,2], [1,3])[:,:,:,:]

        except:
            print("File not Found, going on")
        
        

        for rep in range(reps):
            

            
            
            
            rep = 0
            vel_rep = vel[rep,:,:,:]
            pos_rep = pos[rep,:,:,:]


            #print(np.shape(vel_rep))


            pol = []
            IID = []
            CND = []
            predTime = 2
            for ii in range(1300,args["steps"]-predTime):
                pol.append(calc_order(vel_rep[ii,:nprey,:]))
                pos_calc = pos_rep[ii,:nprey,:]
                dist = scipy.spatial.distance.cdist(pos_calc,pos_calc)
                IID.append(dist.sum() / (nprey * (nprey - 1)))
                dist[dist==0]=100
                CND.append(np.mean(dist.min(axis=1)))
                
                
                
            pol_reps.append(np.mean(pol))
            IID_reps.append(np.mean(IID))
            CND_reps.append(np.mean(CND))
            
            
        time_last = time_now
            
        time_now = datetime.datetime.now()
            
        time_diff = np.round((time_now-time_last).total_seconds(),2)
            
        time_elapsed += time_diff

        progress = rep+reps*(j+i*steps)
            
        time_finish = (time_elapsed/progress) * (total - progress)
            
        print("progress: "+str(np.round(100*progress/total,2))+" %, time running: "+str(np.round(time_elapsed,2))+" s, est. finish: "+str(np.round(time_finish/60,2))+" min.")
            
            
        pol_scan[i,j] = np.mean(pol_reps)
        IID_scan[i,j] = np.mean(IID_reps)
        CND_scan[i,j] = np.mean(CND_reps)
            
        
np.savetxt(str(saveLoc)+""+str(saveName)+"pol_"+str(paraChange1_name)+"_"+str(paraChange2_name)+".csv", pol_scan, delimiter=",")
np.savetxt(str(saveLoc)+""+str(saveName)+"IID_"+str(paraChange1_name)+"_"+str(paraChange2_name)+".csv", IID_scan, delimiter=",")
np.savetxt(str(saveLoc)+""+str(saveName)+"CND_"+str(paraChange1_name)+"_"+str(paraChange2_name)+".csv", CND_scan, delimiter=",")
 


File not Found, going on


NameError: name 'vel' is not defined

In [145]:
dist[dist==0]=100

In [148]:
df = pd.DataFrame(dist)

# Find the 150 smallest values in each row
smallest = df.apply(lambda x: heapq.nsmallest(1, x), axis=1)
smallest_df = pd.DataFrame(smallest.values.tolist())

a = smallest_df.to_numpy()

np.mean(a)

0.6706723040175078

In [4]:
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import h5py
import glob
sys.path.insert(1, '/home/lars/predatorprey-1/predprey-staticnw/')
from animateSwarm import AnimateTools as at
from TsTools import general as gen
from pathlib import Path
import pickle
from functools import partial
from vmodel import geometry as vgeom
from vmodel import plot
from vmodel.util import color as vcolor
import math

def filter_front_target(pos, vel, target):
    vel_self = vel[target]
    pos_self = pos[target]
    pos = np.delete(pos, target, axis=0)
    pos = pos - pos_self
    
    
    
    
    out_idx = []
    for i in range(len(pos)):
        
        ort_pv = [0,0]
        ort_pv[0] = -vel_self[1]
        ort_pv[1] = vel_self[0]
        
        r_pi = np.array(pos[i])
        if (-r_pi[0] * ort_pv[1] + r_pi[1] * ort_pv[0] < 0):
            out_idx.append(i)
            
    return out_idx
        
def pavas2colors(pavas):
    if np.std(pavas) > 1e-5:
        colors = np.squeeze(pavas)
        colors -= colors.min()
        colors /= colors.max()
    else:
        colors = 'k'
    return colors




#file_h5 = str(file)+".h5"
#file_dat = str(file)+".dat"



mode = 'gif' # 'normal', 'pictures', 'movie', 'gif'
fps = 15
dpi = 200
NsamShow = 4
sizePrey = 1/4
sizePred = sizePrey * 2
cmap = plt.get_cmap('coolwarm') # alternatives 'bwr', 'Reds'




# Construct output file name
out_str = "/home/lars/vmodel_output/vmodel_output/npPred/"
out_str = "/home/lars/vmodel_output/"
args_def = {
'nprey': 100,
'npred': 5,
'frange': 10,
'fstr': 5,
'visPred': 120,
'visPrey': 330,
'astr': 3,
'dphi': 0.2,
'repPrey': 1,
'repRadPrey': 1,
'repPred': 1,
'repRadPred': 20,
'attPrey': 2,
'attRadPrey': 3,
'repCol': 1000,
'hstr': 1,
'steps': 12000,
    }

args_def = {
'nprey': 100,
'npred': 1,
'frange': 10,
'fstr': 5,
'visPred': 300.0,
'visPrey': 330,
'astr': 3,
'dphi': 0.2,
'repPrey': 3,
'repRadPrey': 1.5,
'repPred': 1,
'repRadPred': 20,
'attPrey': 3,
'attRadPrey': 1.5,
'repCol': 10000000,
'hstr': 1,
'steps': 1200,
    }





#list_args[1]["fstr"] = 20
#list_args[2]["fstr"] = 50

#list_args[3]["frange"] = 5
#list_args[4]["frange"] = 20

#list_args[5]["astr"] = 1
#list_args[6]["astr"] = 10

#list_args[0]["hstr"] = 5.0
#list_args[1]["hstr"] = 20.0

#list_args[2]["visPred"] = 60
#list_args[3]["visPred"] = 30

#list_args[4]["visPrey"] = 200
#list_args[5]["visPrey"] = 100

#var_par = ["fstr", "fstr", "fstr", "frange", "frange", "astr", "astr", "visPred", "visPred", "visPrey", "visPrey"]
#var_val = [5, 20, 50, 5, 20, 1, 10, 60, 30, 200, 100]




var_par = ["dphi", "dphi", "repRadPred", "repRadPred", "repRadPred", 'repPred', 'repPred', 'repPred','visPred', 'visPred', 'visPred' ]
var_val = [0.01, 1.0, 5.0, 10.0, 25.0, 0.5, 2.0, 5.0, 90.0, 180.0, 300.0]



var_par = ["npred"]
var_val = [1]

runs = len(var_par)
#runs = 1

list_args = []
for i in range(runs):
    list_args.append(args_def.copy())

if len(var_par) != len(var_val):
    sys.exit( "Lists do not match!")


for i in range(runs):
    
    args = list_args[i]
    args[var_par[i]] = var_val[i]
    
    npred = args["npred"]
    nprey = args["nprey"]
    pred_visangle = 2*math.pi*args["visPred"]/360
    prey_visangle = 2*math.pi*args["visPrey"]/360



    args_str = '_'.join(f'{k}_{v}' for k, v in args.items())
    file_h5 = f'{out_str}_{args_str}.states.nc'
    file_h5 = "/home/lars/vmodel_output/_nprey_100_npred_1_frange_10_fstr_50.0_visPred_300.0_visPrey_330_astr_3_dphi_0.2_repPrey_3_repRadPrey_1.5_repPred_1_repRadPred_20_attPrey_3_attRadPrey_1.5_repCol_10000000_hstr_1_steps_1800.states.nc"



    #file_h5 = "/home/lars/vmodel/output/state.nc"
    name = "/home/lars/vmodel_output/testingPoly2_change_"+str(var_par[i])+"="+str(var_val[i])+"_"+args_str
    print(name)
    reps = [0,1,2,3,4]
    for rep in reps:
        with h5py.File(file_h5) as fh5:


            pos = np.moveaxis(np.array(fh5['/position']), [3,2], [1,3])[0,:,:,:]
            vel = np.moveaxis(np.array(fh5['/velocity']), [3,2], [1,3])[0,:,:,:]
            vis = np.array(fh5['/visibility'])

/home/lars/vmodel_output/testingPoly2_change_npred=1_nprey_100_npred_1_frange_10_fstr_5_visPred_300.0_visPrey_330_astr_3_dphi_0.2_repPrey_3_repRadPrey_1.5_repPred_1_repRadPred_20_attPrey_3_attRadPrey_1.5_repCol_10000000_hstr_1_steps_1200


In [10]:
import vmodel
import os
import numpy as np

out_str = "/home/lars/vmodel_output/vmodel_output/npPred/"
args_def = {
'nprey': 100,
'npred': 1,
'frange': 10,
'fstr': 0.0,
'visPred': 300.0,
'visPrey': 330,
'astr': 3,
'dphi': 0.2,
'repPrey': 1,
'repRadPrey': 1.5,
'repPred': 1,
'repRadPred': 20,
'attPrey': 3,
'attRadPrey': 1.5,
'repCol': 10000000,
'hstr': 1,
'steps': 12000,
    }



paraChange1_name = "--alignment-strength"
paraChange2_name = "--dphi"
steps = 20
reps = 30
saveLoc = "/home/lars/vmodel_output/test2"
saveLoc = "/extra2/knopf/vmodel_output/noPred_noMultCol"

paraChange1_val = np.linspace(0,3,steps)
paraChange2_val = np.linspace(0,1,steps)


In [12]:
paraChange1_val

array([0.        , 0.15789474, 0.31578947, 0.47368421, 0.63157895,
       0.78947368, 0.94736842, 1.10526316, 1.26315789, 1.42105263,
       1.57894737, 1.73684211, 1.89473684, 2.05263158, 2.21052632,
       2.36842105, 2.52631579, 2.68421053, 2.84210526, 3.        ])

In [18]:
np.shape(pos)

(1301, 101, 2)